In [1]:
import tensorflow as tf
from keras.models import Sequential
from keras import layers
from keras.callbacks import EarlyStopping
import keras_tuner as kt

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import fetch_california_housing

In [2]:
data = fetch_california_housing()
X, y = data.data, data.target

In [3]:
print(data.feature_names)
print(data.target_names)

['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']
['MedHouseVal']


In [4]:
print(X.shape)
print(y.shape)

(20640, 8)
(20640,)


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [7]:
def build_model(hp):
    model = Sequential()
    model.add(layers.Input(shape=(X_train.shape[1], )))
    
    for i in range(hp.Int('num_layers', 1, 5)):
        model.add(layers.Dense(
			units=hp.Int('units', min_value=32, max_value=256, step=32),
			activation=hp.Choice('activation', values=['relu', 'tanh']),
		))
        model.add(layers.Dropout(hp.Float('dropout', 0, 0.5, step=0.1)))
        
    model.add(layers.Dense(1, activation='sigmoid'))
    
    model.compile(optimizer=hp.Choice('optimizer', values =['adam', 'sgd', 'rmsprop', 'adadelta']), loss='mse', metrics=['mae'])
    
    return model

In [11]:
tuner = kt.Hyperband(build_model, objective='val_mae', max_epochs=5, factor=3, directory='my_dir', project_name='Hyperband_Tuner')

Reloading Tuner from my_dir\Hyperband_Tuner\tuner0.json


In [12]:
# early stopping
stop_early = EarlyStopping(monitor='val_loss', patience=5)

In [13]:
tuner.search(
	X_train, y_train, 
	epochs=5,
	validation_data=(X_test, y_test),
	callbacks=[stop_early]
)

Trial 8 Complete [00h 00m 05s]
val_mae: 1.1398742198944092

Best val_mae So Far: 1.1398742198944092
Total elapsed time: 00h 01m 11s


In [14]:
best_params = tuner.get_best_hyperparameters(num_trials=1)[0]
print(best_params.values)

{'num_layers': 2, 'units': 224, 'activation': 'tanh', 'dropout': 0.0, 'optimizer': 'adam', 'tuner/epochs': 5, 'tuner/initial_epoch': 0, 'tuner/bracket': 0, 'tuner/round': 0}


In [15]:
best_model = tuner.get_best_models(num_models=1)[0]

C:\Users\admin\AppData\Roaming\Python\Python312\site-packages\keras\src\saving\saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [ ]:
history = best_model.fit(X_train, y_train, initial_epoch=5, epochs=15, validation_data=(X_test, y_test), callbacks=[stop_early])

Epoch 7/15
516/516 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 2.4861 - mae: 1.1544 - val_loss: 2.4234 - val_mae: 1.1407
Epoch 8/15
516/516 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 2.4858 - mae: 1.1546 - val_loss: 2.4234 - val_mae: 1.1408
Epoch 9/15
516/516 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 2.4867 - mae: 1.1547 - val_loss: 2.4234 - val_mae: 1.1408
Epoch 10/15
516/516 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 2.4859 - mae: 1.1546 - val_loss: 2.4234 - val_mae: 1.1408
Epoch 11/15
516/516 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 2.4858 - mae: 1.1546 - val_loss: 2.4234 - val_mae: 1.1408
Epoch 12/15
516/516 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 2.4858 - mae: 1.1546 - val_loss: 2.4234 - val_mae: 1.1408
Epoch 13/15
516/516 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 2.4858 - mae: 1.1546 - val_loss: 2.4234 - val_mae: 1.1408
Epoch 14/15
516/516 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 2.4862 - mae: 1.1547 - val_loss: 2.4235 - val_mae: 1.1408


In [17]:
mse, mae = best_model.evaluate(X_test, y_test)
print(f"MSE : {mse}, MAE : {mae}")

129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 909us/step - loss: 2.4235 - mae: 1.1408
MSE : 2.423466444015503, MAE : 1.1407756805419922
